In [1]:
import Constants
from  openai import OpenAI
import os
import numpy as np
import pandas as pd
import time

import gspread
import gspread_dataframe as gd
import gspread_formatting as gf
from gspread_formatting import cellFormat, color, textFormat


## Manual Phrase Generator

In [5]:
from main.phrase_generator import *

prompt = get_prompt_to_gen_phrases(
            situation = "Going to the library",
            num_phrases = 5,
            complexity = "Medium",
            )

In [7]:
translation_model="gpt-4o"
temp=0.7 

translation_response =  get_completion(
        prompt = prompt,
        model=translation_model, 
        temperature=temp)


In [26]:
def parse_translation_response(
        content: str, 
        ffill_cols: List[str] = None,
        date_col: List[str] = None
        ) -> pd.DataFrame:
    '''
    Parse the table response from OpenAI into a pandas DataFrame
    '''
    # Using StringIO to treat the text as a file-like object for pandas
    data = StringIO(content)

    # Read the table into a pandas DataFrame
    df = pd.read_csv(data, delimiter='|',  engine='python')

    # Cleaning the DataFrame by stripping leading/trailing whitespaces from column names and data
    df.columns = df.columns.str.strip()
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    
    # Remove rows with dashes
    col_name = df.select_dtypes(include=['object']).columns[0]
    df = df.loc[~df[col_name].str.contains('--')]

    col_to_keep = [col for col in df if 'Unnamed' not in col]
    df = df[col_to_keep]

    if ffill_cols:
        for col in ffill_cols:
            df[col] = df[col].replace('', pd.NA).ffill()

    if date_col:
        for col in date_col:
            df[col] = datetime.now().strftime("%Y-%m-%d")

    return df

In [27]:
df = parse_translation_response(translation_response.choices[0].message.content)

Index(['Line', 'Pinyin', 'Meaning', 'Response', 'Response Pinyin',
       'Response Meaning', 'Complexity', 'Category'],
      dtype='object')

In [ ]:

newwords_df = (
    parse_translation_response(
        translation_response.choices[0].message.content,
        ffill_cols = ['Word', 'Pinyin', 'Pinyin Simplified', 'Type'],
        date_col = ['Added Date']
        )
    )

## Generate words rarity classification
rarity_response = (
    get_completion(
        prompt=get_prompt_for_rarity_classification(word_list), model=rarity_model, temperature=temp))

word_rarity_df = parse_translation_response(rarity_response.choices[0].message.content)
newwords_df = pd.merge(newwords_df, word_rarity_df, on='Word', how='left')

if not (replace_new_words) and len(self.new_words_df) > 0:
    orig_new_words_df = self.new_words_df.copy()

    if len(orig_new_words_df) > 0:
        orig_new_words_df = orig_new_words_df.loc[~orig_new_words_df.Word.isin(newwords_df.Word)]
    self.new_words_df = pd.concat([orig_new_words_df, newwords_df])